<a href="https://colab.research.google.com/github/shizoda/education/blob/main/info_theory/communication_sender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 通信シミュレーション（送信側）

このプログラムは，情報理論における符号化を学ぶため，雑音のある通信路を経由してメッセージを送信するためのプログラムです．具体的には，ハフマン符号化とハミング符号化を行い，雑音のある経路からデータの送信を行います．また，今回は情報のやりとりをGoogle Driveを経由して行います．

## 準備

まず，Google Driveをマウントします．この際，Googleからの警告が出るので，マウントの許可を行ってください．このとき，送信するデータはMyDriveの下に自動で追加されるinfo-theorフォルダに保存されます．

使用するライブラリはGithub上に存在するため，wgetで取得し，importを行います．

In [ ]:
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running on Google Colab")
    drive.mount('/content/drive')
    saved_dir = '/content/drive/MyDrive/info-theory'
except ImportError:
    IN_COLAB = False
    print("Not running on Google Colab")
    saved_dir = '//nas/info-theory'

import os
os.makedirs(saved_dir, exist_ok=True)
print(f"Saved directory is set to: {saved_dir}")

import sys
sys.path.append(saved_dir)

# GitHubリポジトリのURL
!wget -q -O {saved_dir}/communication_lib.py https://raw.githubusercontent.com/shizoda/education/main/info_theory/communication_lib.py

# 必要なPythonファイルをインポートする
from communication_lib import *

Running on Google Colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved directory is set to: /content/drive/MyDrive/info-theory


## メッセージ

送り主，宛先，メッセージ本文を指定します．以下のコードでは，aliceがデータを送信し，bobが受信します．

なお，サンプルの英文は "e"，和文は「静」や「岡」が多用された文になっています．これはハフマン符号化を効果的に行うためです．

In [ ]:
message_from = "alice"
message_to   = "bob"

message = "Everyone expects excellent experiences every evening. Each evening, Ellie eagerly eats exquisite edibles, enjoying every essence. Evenings envelop Ellie, embracing endless energy. Ellie engages effortlessly, expressing emotions endlessly. Every encounter elevates Ellie's enthusiasm, enriching every experience eternally."
# message = "静岡県は静かで美しい場所です。静岡市は県の中心です。静岡には静岡茶や静岡おでんなどの名物があります。静かな山々や岡の風景も楽しめます。"
print(f"メッセージ: {message}")

メッセージ: Everyone expects excellent experiences every evening. Each evening, Ellie eagerly eats exquisite edibles, enjoying every essence. Evenings envelop Ellie, embracing endless energy. Ellie engages effortlessly, expressing emotions endlessly. Every encounter elevates Ellie's enthusiasm, enriching every experience eternally.


## 符号化・送信の実施

以下ではハフマン符号化，ハミング符号化，雑音のある通信路からの送信を行います．
次の説明を参考に，ハフマン符号化，ハミング符号化，雑音のある通信路からの送信を行うセルの順番を正しく入れ替えて，データを送信してください．

### ハフマン符号

ハフマン符号は，可逆なデータ圧縮のためのアルゴリズムの1つです．基本原理としては，出現頻度の高い文字に対しては短い符号を，頻度の高い文字に対しては長い符号を振り分けることで，すべての文字に同じ長さの符号を振り分けるよりも，全体の符号化後のデータサイズを最小にしています．

実際にハフマン符号化を行う際には，文字の出現頻度によって，ハフマン木と呼ばれる木構造を作成します．この木では，葉に近いノードほど出現頻度が低く，根に近いノードほど出現頻度が高くなります．また，自分よりも根に近いノードの数が多ければ多いほど，符号の長さが長くなります．以下のコードでは，ハフマン符号化を行った際に用いたハフマン木も出力されます．

### ハミング符号

ハミング符号は，データ通信などで誤り検出と訂正を行うための誤り訂正符号の一種です．これにより，通信中や保存中に発生する単一ビット誤りを検出し，訂正することが出来ます．ハミング符号は，データビットとパリティビットを組み合わせて構成されます．データビットは送信したいデータが保存され，パリティビットは特定のデータビットの組み合わせに基づいて計算され，データビットに誤りが生じた際に，その位置を特定し，訂正するために使用されます．

ハミング符号は，単一ビットエラーには対応できますが，複数のビットエラーには対応できないため，実用上はより高度なエラー訂正技術と組み合わせて使用されます．

以下のコードでは，単一ビットエラーのみが発生することを仮定し，ハミング符号化を行います．

### 雑音のある通信路からの送信

実際にデータをやりとりする際には，

In [ ]:
print("△△△符号化器：ハフマン符号")
encoded_text, codebook, padding_length = huffman_encoding(message)

△△△符号化器：ハフマン符号

Character Codes: E:69 v:118 e:101 r:114 y:121 o:111 n:110 e:101  :32 e:101 x:120 p:112 e:101 c:99 t:116 s:115  :32 e:101 x:120 c:99 e:101 l:108 l:108 e:101 n:110 t:116  :32 e:101 x:120 p:112 e:101 r:114 i:105 e:101 n:110 c:99 e:101 s:115  :32 e:101 v:118 e:101 r:114 y:121  :32 e:101 v:118 e:101 n:110 i:105 n:110 g:103 .:46  :32 E:69 a:97 c:99 h:104  :32 e:101 v:118 e:101 n:110 i:105 n:110 g:103 ,:44  :32 E:69 l:108 l:108 i:105 e:101  :32 e:101 a:97 g:103 e:101 r:114 l:108 y:121  :32 e:101 a:97 t:116 s:115  :32 e:101 x:120 q:113 u:117 i:105 s:115 i:105 t:116 e:101  :32 e:101 d:100 i:105 b:98 l:108 e:101 s:115 ,:44  :32 e:101 n:110 j:106 o:111 y:121 i:105 n:110 g:103  :32 e:101 v:118 e:101 r:114 y:121  :32 e:101 s:115 s:115 e:101 n:110 c:99 e:101 .:46  :32 E:69 v:118 e:101 n:110 i:105 n:110 g:103 s:115  :32 e:101 n:110 v:118 e:101 l:108 o:111 p:112  :32 E:69 l:108 l:108 i:105 e:101 ,:44  :32 e:101 m:109 b:98 r:114 a:97 c:99 i:105 n:110 g:103  :32 e:101 n:110 d:100 l:108 e

In [ ]:
print("△△△符号化器：ハミング符号")
encoded_data = hamming_encode(encoded_text)

通信路符号化器：ハミング符号
Hamming Input  : 0100 0100 1000 1111 1110 0011 1100 1110 0001 1001 1011 1100 01.. .
Hamming Encoded: 1001100 1001100 1110000 1111111 0010110 1000011 0111100 0...


In [ ]:
# 相手に届くビットには雑音が載る（一部の 0 / 1 がひっくり返る）
print("雑音のある通信路から送信")
send_message(saved_dir, message_from, message_to, encoded_data, codebook, padding_length)

雑音のある通信路から送信

雑音のある通信路
Transmitted    : 1000100 1001000 1110000 1111111 0010111 1000011 0110100 0...
